# 01 Hello World (Experiment)
Read first: `examples/01_hello_world.py`

Minimal load -> compose -> execute loop. Use this after reading the script to tweak provider/prompt and see the response live.


## Select a provider
Set `PROVIDER_KEY` to `sonnet`, `opus`, `gpt`, or `gpt-codex` and run the next cell.
The helper will set env vars (prompting if missing) and print the selection.


In [49]:
# Choose your provider key (sonnet | opus | gpt | gpt-codex)
PROVIDER_KEY = "gpt"

In [50]:
# Resolve foundation/provider paths and ensure env var
import importlib.util
import os
from pathlib import Path


def find_repo_root() -> Path:
    candidates = [Path.cwd(), Path.cwd().parent, Path.cwd().parent.parent]
    for p in candidates:
        if (p / "utils" / "providers.py").exists():
            return p
    return Path.cwd()


ROOT = find_repo_root()
providers_path = ROOT / "utils" / "providers.py"
spec = importlib.util.spec_from_file_location("providers_helper", providers_path)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)
select_provider = module.select_provider
print_provider_menu = module.print_provider_menu

FOUNDATION_PATH, PROVIDER_PATH, REQUIRED_ENV = select_provider(PROVIDER_KEY, foundation="minimal")
print_provider_menu(PROVIDER_KEY)
print(f"Selected: {PROVIDER_KEY} -> {PROVIDER_PATH.name}")
env_set = "Yes" if os.environ.get(REQUIRED_ENV) else "No"
print(f"Required env: {REQUIRED_ENV} (set? {env_set})")

Providers (set PROVIDER_KEY):
     sonnet: anthropic-sonnet.yaml
     opus: anthropic-opus.yaml
  -> gpt: openai-gpt.yaml
     gpt-codex: openai-gpt-codex.yaml
Selected: gpt -> openai-gpt.yaml
Required env: OPENAI_API_KEY (set? Yes)


In [51]:
# Optional: set API key here (safer: export in terminal)
import getpass
import os

if not os.getenv(REQUIRED_ENV):
    os.environ[REQUIRED_ENV] = getpass.getpass(f"Enter {REQUIRED_ENV}: ")
    print(f"Set {REQUIRED_ENV} for this kernel session.")
else:
    print(f"{REQUIRED_ENV} already set.")

OPENAI_API_KEY already set.


In [52]:
# Parameters (edit me)
PROMPT = "Write a Python function to check if a number is prime. Include docstring and type hints."

In [53]:
# Run the agent
from amplifier_foundation import load_bundle


async def run(prompt: str):
    foundation = await load_bundle(str(FOUNDATION_PATH))
    provider = await load_bundle(str(PROVIDER_PATH))
    composed = foundation.compose(provider)
    prepared = await composed.prepare()
    session = await prepared.create_session()
    async with session:
        return await session.execute(prompt)


response = await run(PROMPT)
print(response)

No approval system provided - approval requests will fail
No display system provided - hook messages will be logged only


```python
from __future__ import annotations

import math


def is_prime(n: int) -> bool:
    """
    Return True if `n` is a prime number, otherwise False.

    A prime number is an integer greater than 1 that has no positive divisors
    other than 1 and itself.

    Args:
        n: Integer to test for primality.

    Returns:
        True if `n` is prime, False otherwise.
    """
    if n <= 1:
        return False
    if n <= 3:
        return True  # 2 and 3 are prime
    if n % 2 == 0 or n % 3 == 0:
        return False

    # Check possible divisors up to sqrt(n), skipping even numbers.
    limit = math.isqrt(n)
    d = 5
    while d <= limit:
        if n % d == 0 or n % (d + 2) == 0:
            return False
        d += 6  # tests 6k±1
    return True
```


### Try this
- Swap `PROVIDER_PATH` to another file in `providers/`.
- Change `PROMPT` to see how the response shifts.
- If you lack the env var, set it in the terminal and re-run.
